In [15]:
import sys

import geopandas as gpd
import movingpandas as mpd
import pandas as pd
from movingpandas import Trajectory

sys.path.append('/home/diviloper/MobilityDB/PyMEOS/')
from pymeos_cffi.functions import meos_initialize
from pymeos import TGeogPointSeq, TGeogPointInst

meos_initialize()

In [16]:
def create_point(row) -> TGeogPointInst:
    return TGeogPointInst(string=f"{row['geometry']}@{row.name}")


def extract_wkt_from_traj(trajectory: Trajectory) -> str:
    res = "["
    for row in trajectory.df.iloc():
        item = row.geometry.wkt + '@' + row.name.__str__()
        res += item + ','
    res = res[:-1] + ']'
    return res


def create_sequence(trajectory: Trajectory) -> TGeogPointSeq:
    wkt = extract_wkt_from_traj(trajectory)
    return TGeogPointSeq(string=wkt)

### Read AIS CSV

In [17]:
pdf = pd.read_csv('./aisinput.csv')
gdf = gpd.GeoDataFrame(pdf.drop(['latitude', 'longitude'], axis=1),
                       geometry=gpd.points_from_xy(pdf.longitude, pdf.latitude))
gdf.head()

,t,mmsi,sog,geometry
0,2021-01-08 00:00:00,265513270,0.0,POINT (12.27239 57.05900)
1,2021-01-08 00:00:01,219027804,0.0,POINT (11.86628 55.94244)
2,2021-01-08 00:00:01,265513270,0.0,POINT (12.27239 57.05900)
3,2021-01-08 00:00:03,219027804,0.0,POINT (11.86628 55.94244)
4,2021-01-08 00:00:04,265513270,0.0,POINT (12.27239 57.05900)


In [28]:
ships = mpd.TrajectoryCollection(gdf, 'mmsi', t='t')

/home/diviloper/.local/share/virtualenvs/PyMEOS-aif996WK/lib/python3.8/site-packages/movingpandas/trajectory.py:123: MissingCRSWarning: Trajectory generated without CRS. Computations will use Euclideandistances.
  warnings.warn(
/home/diviloper/.local/share/virtualenvs/PyMEOS-aif996WK/lib/python3.8/site-packages/movingpandas/trajectory.py:123: MissingCRSWarning: Trajectory generated without CRS. Computations will use Euclideandistances.
  warnings.warn(
/home/diviloper/.local/share/virtualenvs/PyMEOS-aif996WK/lib/python3.8/site-packages/movingpandas/trajectory.py:123: MissingCRSWarning: Trajectory generated without CRS. Computations will use Euclideandistances.
  warnings.warn(
/home/diviloper/.local/share/virtualenvs/PyMEOS-aif996WK/lib/python3.8/site-packages/movingpandas/trajectory.py:123: MissingCRSWarning: Trajectory generated without CRS. Computations will use Euclideandistances.
  warnings.warn(
/home/diviloper/.local/share/virtualenvs/PyMEOS-aif996WK/lib/python3.8/site-packages

In [33]:
ship = ships.trajectories[0]
ship.df.head()

,mmsi,sog,geometry
t,,,
2021-01-08 00:00:05,219001559,0.1,POINT (9.97551 57.59224)
2021-01-08 00:00:11,219001559,0.1,POINT (9.97551 57.59226)
2021-01-08 00:00:18,219001559,0.1,POINT (9.97550 57.59226)
2021-01-08 00:00:24,219001559,0.1,POINT (9.97550 57.59226)
2021-01-08 00:00:31,219001559,0.1,POINT (9.97549 57.59225)


In [34]:
ship.df['MEOS Point'] = ship.df.apply(create_point, axis=1)
ship.df.head()

,mmsi,sog,geometry,MEOS Point
t,,,,
2021-01-08 00:00:05,219001559,0.1,POINT (9.97551 57.59224),POINT(9.976 57.592)@2021-01-08 00:00:05+01
2021-01-08 00:00:11,219001559,0.1,POINT (9.97551 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:11+01
2021-01-08 00:00:18,219001559,0.1,POINT (9.97550 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:18+01
2021-01-08 00:00:24,219001559,0.1,POINT (9.97550 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:24+01
2021-01-08 00:00:31,219001559,0.1,POINT (9.97549 57.59225),POINT(9.975 57.592)@2021-01-08 00:00:31+01


In [35]:
ship.add_speed(overwrite=True)

In [36]:
ship.add_distance(overwrite=True)
sum(ship.df[ship.distance_col_name])

0.16303366821181925

In [37]:
sequence = create_sequence(ship)
sequence.distance

5202.381848699494

In [32]:
ship.df

,mmsi,sog,geometry,MEOS Point,speed,distance
t,,,,,,
2021-01-08 00:00:01,219027804,0.0,POINT (11.86628 55.94244),POINT(11.866 55.942)@2021-01-08 00:00:01+01,0.000000,0.000000
2021-01-08 00:00:03,219027804,0.0,POINT (11.86628 55.94244),POINT(11.866 55.942)@2021-01-08 00:00:03+01,0.000000,0.000000
2021-01-08 00:00:05,219027804,0.0,POINT (11.86628 55.94244),POINT(11.866 55.942)@2021-01-08 00:00:05+01,0.000000,0.000000
2021-01-08 00:00:07,219027804,0.0,POINT (11.86628 55.94244),POINT(11.866 55.942)@2021-01-08 00:00:07+01,0.000001,0.000002
2021-01-08 00:00:09,219027804,0.0,POINT (11.86628 55.94244),POINT(11.866 55.942)@2021-01-08 00:00:09+01,0.000000,0.000000
...,...,...,...,...,...,...
2021-01-08 23:59:48,219027804,0.0,POINT (11.86628 55.94246),POINT(11.866 55.942)@2021-01-08 23:59:48+01,0.000000,0.000000
2021-01-08 23:59:51,219027804,0.0,POINT (11.86628 55.94246),POINT(11.866 55.942)@2021-01-08 23:59:51+01,0.000000,0.000000
2021-01-08 23:59:53,219027804,0.0,POINT (11.86628 55.94246),POINT(11.866 55.942)@2021-01-08 23:59:53+01,0.000000,0.000000
